In [ ]:
import pandas as pd
import os
forder_train = "/content/drive/MyDrive/BioCreativeVIII/BioREx/datas/Bio_Competition_dataset/train"
list_file = os.listdir(forder_train)  
# Initialize data collection list
list_data_NER = []
list_data_RE = []
text_t_in_documents = {} 
text_a_in_documents = {}
for file_path in list_file: 
    file_path = os.path.join(forder_train, file_path)  
# Open and process the file
    with open(file_path, 'r') as file:

        for line in file:
            fields = line.strip().split('\t')
            if len(fields) == 6:
                list_data_NER.append([fields[0],fields[1], fields[2], fields[3], fields[4], fields[5]])
            elif len(fields) == 1 and '|t|' in fields[0]: 
                position_t = fields[0].find('|t|')
                id_document = fields[0][:position_t]
                text_t_in_documents.update({id_document :fields[0][position_t+3:]})
            elif len(fields) == 1 and '|a|' in fields[0]:
                position_a = fields[0].find('|a|')
                id_document = fields[0][:position_a] 
                text_a_in_documents.update({id_document :fields[0][position_a+3:]}) 
            elif len(fields) == 5: 
                list_data_RE.append([fields[0],fields[1], fields[2], fields[3], fields[4]])
    data_frame_RE = pd.DataFrame(list_data_RE, columns=["doc_id", "relation", "id_identifier_1", "id_identifier_2", "novel_cls"])       
    data_frame_NER = pd.DataFrame(list_data_NER, columns=["doc_id", "offset_1", "offset_2", "entity", "identifier", "id_identifier"]) 

In [ ]:
text_in_documents={}
for key in text_t_in_documents.keys(): 
    full_text = text_t_in_documents[key] + " " + text_a_in_documents[key]
    text_in_documents.update({key:full_text}) 
data_frame_NER['full_document'] = data_frame_NER['doc_id'].map(text_in_documents)

In [ ]:
data_frame_NER_work = data_frame_NER.copy()
data_frame_NER_work

In [ ]:
# Assuming df is your DataFrame
def extract_entity(row):
    return row['full_document'][int(row['offset_1']):int(row['offset_2'])]

data_frame_NER_work['entity1_offset'] = data_frame_NER_work.apply(extract_entity, axis=1)
data_frame_NER_work 

In [ ]:
import pandas as pd

def extract_full_text(row):
    start_index = int(row['offset_1'])
    end_index = int(row['offset_2'])
    full_document = row['full_document']
    
    # Find the position of the first '.' after end_index
    end_period_index = full_document.find('.', end_index)
    
    # Find the position of the last '.' before start_index
    last_period_index = full_document.rfind('.', 0, start_index)
    
    if last_period_index != -1:
        return full_document[last_period_index + 1:end_period_index + 1]
    else:
        #return full_document[start_index:end_period_index + 1]
        return full_document[0:end_period_index + 1]

# Assuming df is your DataFrame
data_frame_NER_work['full_sentence'] = data_frame_NER_work.apply(extract_full_text, axis=1)

# Print the updated DataFrame
data_frame_NER_work["full_sentence"].iloc[1000]


In [ ]:
data_frame_NER_work["entity"].iloc[1000]
data_frame_NER_clean = data_frame_NER_work.drop(['offset_1', 'offset_2', "full_document", "entity1_offset"], axis=1)
data_frame_NER_clean

In [ ]:
data_frame_NER_clean.drop_duplicates(subset=['entity', 'full_sentence', 'doc_id'], inplace=True)
data_frame_NER_clean

In [ ]:
data_test_1 = data_frame_NER_clean.rename(columns={
  "entity":"entity_1", 
  "identifier":"identifier_1",
  "id_identifier":"id_identifier_1",
  "full_sentence":"full_sentence_1"
})

data_test_2 = data_frame_NER_clean.rename(columns={
  "entity":"entity_2", 
  "identifier":"identifier_2",
  "id_identifier":"id_identifier_2",
  "full_sentence":"full_sentence_2"
})

In [ ]:

merged_df = pd.merge(data_frame_RE, data_test_1, on=['doc_id', 'id_identifier_1'], how='inner')
merged_df 

In [ ]:
merged_df_2 = pd.merge(merged_df, data_test_2, on=['doc_id', 'id_identifier_2'], how='inner')

In [ ]:
merged_df_2.drop(['doc_id', 'id_identifier_2', "id_identifier_1"], axis=1, inplace=True)

In [ ]:

data1_x = merged_df_2.rename(columns={
  "entity_1":"entity_x", 
  "identifier_1":"identifier_x",
  "id_identifier_1":"id_identifier_x",
  "full_sentence_1":"full_sentence_x"
})


data2_1 = data1_x.rename(columns={
  "entity_2":"entity_1", 
  "identifier_2":"identifier_1",
  "id_identifier_2":"id_identifier_1",
  "full_sentence_2":"full_sentence_1"
})

datax_2 = data2_1.rename(columns={
  "entity_x":"entity_2", 
  "identifier_x":"identifier_2",
  "id_identifier_x":"id_identifier_2",
  "full_sentence_x":"full_sentence_2"
})
datax_2

In [ ]:
merged_df_2

In [ ]:
combined_df = pd.concat([merged_df_2, datax_2], ignore_index=True)
combined_df.drop_duplicates().to_csv("/content/drive/MyDrive/BioCreativeVIII/BioREx/datas/dataset_train_novel/train_novel_v3.csv")

In [ ]:
combined_df

In [ ]:
combined_df.drop_duplicates()

In [ ]:
data_frame_RE

In [ ]:
doc_id = "BC8_BioRED_Task1_Doc0"
id_identifier = "4312"
doc_id_data = data_frame_NER_clean[data_frame_NER_clean['doc_id'] == doc_id]
doc_id_data_id_identifier = doc_id_data[doc_id_data['id_identifier'] == id_identifier]
sentences = doc_id_data_id_identifier["full_sentence"]

In [ ]:
doc_id_data_id_identifier

In [ ]:
# def extract_entity_from_NER_dataframe_1(doc_id, id_identifier): 
#     doc_id_data = data_frame_NER_clean[data_frame_NER_clean['doc_id'] == doc_id] 
#     doc_id_data_id_identifier = doc_id_data[doc_id_data['id_identifier'] == id_identifier] 
#     doc_id_data_id_identifier.drop(['doc_id','id_identifier'],axis=1, inplace=True) 
#     return doc_id_data_id_identifier.rename(columns={'entity': 'entity_1', 
#                                                      'id_identifier': 'id_identifier_1', 
#                                                      'full_sentence': 'full_sentence_1'
#                                                      }).iloc[0].values
def extract_entity_from_NER_dataframe(doc_id, id_identifier):
    doc_id_data = data_frame_NER_clean[data_frame_NER_clean['doc_id'] == doc_id]
    doc_id_data_id_identifier = doc_id_data[doc_id_data['id_identifier'] == id_identifier]
    
    if not doc_id_data_id_identifier.empty:
        # Check if the resulting DataFrame is not empty
        row = doc_id_data_id_identifier.iloc[0]
        sentences = doc_id_data_id_identifier["full_sentence"]

        return row['entity'], row['identifier'], sentences
    else:
        # Return default values or handle the case when there's no matching row
        return None, None, None


# Gọi hàm extract_entity_from_NER_dataframe
entity, identifier, full_sentence = extract_entity_from_NER_dataframe("BC8_BioRED_Task1_Doc0", "4312")

entity, identifier, full_sentence

In [ ]:
import pandas as pd

# Assuming your DataFrame is named data_frame_RE

# def add_entity_info_to_dataframe_1(row):
#     doc_id = row['doc_id']
#     id_identifier = row['id_identifier_1']
    
#     entity, identifier, full_sentence = extract_entity_from_NER_dataframe(doc_id, id_identifier)
#     if len(full_sentence) > 1:
#         #do this len(full_sentence) time  return pd.Series({'entity_1': entity, 'identifier_1': identifier, 'full_sentence_1': full_sentence}) 
#     else: 
#         return pd.Series({'entity_1': entity, 'identifier_1': identifier, 'full_sentence_1': full_sentence})

def add_entity_info_to_dataframe_1(row):
   doc_id = row['doc_id']
   id_identifier = row['id_identifier_1']

   entity, identifier, full_sentence = extract_entity_from_NER_dataframe(doc_id, id_identifier)

   if len(full_sentence) > 1:
       # Create a list of Series, one for each sentence
       series_list = []
       for i in range(len(full_sentence)):
           series_list.append(pd.Series({'entity_1': entity, 'identifier_1': identifier, 'full_sentence_1': full_sentence[i]}))

       # Concatenate the Series into a DataFrame
       return series_list
    #    return pd.concat(series_list, axis=1)
   else:
      return pd.Series({'entity_1': entity, 'identifier_1': identifier, 'full_sentence_1': full_sentence})


# Apply the function to each row of data_frame_RE
# new_columns = data_frame_RE.apply(add_entity_info_to_dataframe_1, axis=1)
# new_columns
# # Concatenate the new columns to data_frame_RE
# data_frame_RE_merge_1 = pd.concat([data_frame_RE, new_columns], axis=1)
# add_entity_info_to_dataframe_1(data_frame_RE.iloc[0])


In [ ]:
add_entity_info_to_dataframe_1(data_frame_RE.iloc[1])

In [ ]:

def add_entity_info_to_dataframe_2(row):
    doc_id = row['doc_id']
    id_identifier = row['id_identifier_2']
    
    entity, identifier, full_sentence = extract_entity_from_NER_dataframe(doc_id, id_identifier)
    
    return pd.Series({'entity_2': entity, 'identifier_2': identifier, 'full_sentence_2': full_sentence})

# Apply the function to each row of data_frame_RE
new_columns = data_frame_RE_merge_1.apply(add_entity_info_to_dataframe_2, axis=1)

# Concatenate the new columns to data_frame_RE
data_frame_RE_merge_2 = pd.concat([data_frame_RE_merge_1, new_columns], axis=1)

In [ ]:
data_frame_RE_merge_2.to_csv('dataset_train_novel/train_novel.csv')

In [ ]:
data_frame_RE_merge_2

In [ ]:
data_frame_NER['identifier'].value_counts()

In [ ]:
data_frame_RE['novel_cls'].value_counts()

In [ ]:
data_frame_RE['relation'].value_counts()